In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cpu')

In [3]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'GPU를 사용합니다: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')
    print('CPU를 사용합니다.')

CPU를 사용합니다.


In [4]:
import torch
print(torch.__version__)


2.2.0


In [6]:
device = 'cuda:0'

'cuda:0'